In [24]:
import pandas as pd
import urllib, urllib.request
import requests
import time
import json

In [34]:
jauthors = []
jtitles = []
dois = []
jyears = []
jdoctypes = []
jwordcounts = []
journals = []
languages = []

with open('../../LitStudiesJSTOR.jsonl', mode = 'r', encoding = 'utf-8') as f:
    lines = f.readlines()
    ctr = 0
    for l in lines:
        j = json.loads(l)
        ctr += 1
        if ctr % 500 == 1:
            print(ctr)
        if 'creator' in j:
            jauthors.append(j['creator'])
        else:
            jauthors.append("['anonymous']")
        doctype = j['docType'] + " | " + j['docSubType']
        jdoctypes.append(doctype)
        jwordcounts.append(j['wordCount'])
        jyears.append(j['publicationYear'])
        jtitles.append(j['title'])
        journals.append(j['isPartOf'])
        ids = j['identifier']
        thedoi = 'no doi'
        for anid in ids:
            if anid['name'] == 'local_doi':
                thedoi = anid['value']
        dois.append(thedoi)
        languages.append(j['language'])
         
litmeta = pd.DataFrame({'journal': journals, 'year': jyears, 'authors': jauthors,
                       'title': jtitles, 'language': languages,
                       'wordcount': jwordcounts, 'doctype': jdoctypes, 'doi': dois})

1
501
1001
1501
2001
2501
3001
3501
4001
4501
5001
5501
6001
6501
7001
7501
8001
8501
9001
9501
10001
10501
11001
11501
12001
12501
13001
13501
14001
14501
15001
15501
16001
16501
17001
17501
18001
18501
19001
19501
20001
20501
21001
21501
22001
22501
23001
23501
24001
24501
25001
25501
26001
26501
27001
27501
28001
28501
29001
29501
30001
30501
31001
31501
32001
32501
33001
33501
34001
34501
35001
35501
36001
36501
37001
37501
38001
38501
39001
39501
40001
40501
41001
41501
42001
42501
43001
43501
44001
44501
45001
45501
46001
46501
47001
47501
48001
48501
49001
49501
50001
50501
51001
51501
52001
52501
53001
53501
54001
54501
55001
55501
56001
56501
57001
57501
58001
58501
59001
59501
60001
60501
61001
61501
62001
62501
63001
63501
64001
64501
65001
65501
66001
66501
67001
67501
68001
68501
69001
69501
70001
70501
71001
71501


In [36]:
litmeta.head()

,journal,year,authors,title,language,wordcount,doctype,doi
0,The Review of English Studies,1969,[C. F. Williamson],Review Article,[eng],572,article | book-review,10.2307/512209
1,Modern Philology,1979,[David Halliburton],Review Article,[eng],1863,article | book-review,10.2307/437950
2,PMLA,1967,[Jean-Jacques Demorest],Pascal et le déséquilibre,[fre],4999,article | research-article,10.2307/461288
3,The Review of English Studies,2003,[Robert Douglas-Fairhurst],Review Article,[eng],1140,article | book-review,10.2307/3661498
4,The Modern Language Review,1951,[R. Weiss],Review Article,[eng],669,article | book-review,10.2307/3718589


In [37]:
litmeta.to_csv('LitStudiesMetadata.tsv', sep = '\t', index = False)

In [19]:
#takes DOI id, can also use arxiv link or search with title
ids=["10.2307/3713967"]

In [77]:
numrows, numcols = litmeta.shape
newlitmeta = litmeta
newlitmeta = newlitmeta.assign(citation_counts = [0] * numrows)
newlitmeta = newlitmeta.assign(S2titles =  ['NA'] * numrows)
newlitmeta = newlitmeta.assign(S2years = ['no date'] * numrows)

In [86]:
numrows, numcols = litmeta.shape
missed=[]

for idxfloor in range(68700, numrows, 100):
    ids = list(litmeta.iloc[idxfloor : idxfloor + 100, 7])
    idx = idxfloor
    returns = []
    for DOI in ids:
        url="https://api.semanticscholar.org/graph/v1/paper/DOI:"+DOI+"?fields=title,publicationDate,authors,s2FieldsOfStudy,citations,citations.authors"
        response = requests.get(url)
        try:
            resplen = len(response.json())
        except:
            missed.append(url)
            print(DOI)
            idx += 1
            #waits when timeout
            time.sleep(1)
            continue
        i=response.json()
        if 'error' in i:
            idx += 1
            continue
        if 'title' in i:
            title=i["title"]
        else:
            title = 'Title Not Provided'
        if 'publicationDate' in i:
            date=i["publicationDate"]
        else:
            date = '0000-00-01'
        cited_ids = []
        if 'citations' in i:
            for c in i["citations"]:
                cited_ids.append(c["paperId"])
        citation_count=len(cited_ids)
        i['doi'] = DOI
        returns.append(i)
        assert(newlitmeta.at[idx, 'doi'] == DOI)
        newlitmeta.at[idx, 'citation_counts'] = citation_count
        newlitmeta.at[idx, 'S2titles'] = title
        newlitmeta.at[idx, 'S2years'] = date
        idx += 1
        
    with open('S2records4Lit.jsonl', mode = 'a') as f:
        for r in returns:
            f.write(json.dumps(r) + '\n')
    newlitmeta.to_csv('LitStudiesMetaWithS2.tsv', sep = '\t',
                     index = False)
    hundreds = idxfloor / 100
    remainder = idxfloor % 200
    if hundreds < 702 or remainder == 0:
        print(idxfloor)
        

68700
68800
68900
69000
69100
69200
69300
69400
69500
69600
69700
69800
69900
70000
70100
70200
10.2307/3729716
70400
70600
70800
71000
71200
71400
71600
71800


In [87]:
missed

['https://api.semanticscholar.org/graph/v1/paper/DOI:10.2307/3729716?fields=title,publicationDate,authors,s2FieldsOfStudy,citations,citations.authors']

In [72]:
newlitmeta.at[idx, 'doi']

'10.2307/512209'

In [73]:
idx

0

In [74]:
DOI

'10.2307/461288'

In [69]:
ids

['10.2307/512209',
 '10.2307/437950',
 '10.2307/461288',
 '10.2307/3661498',
 '10.2307/3718589',
 '10.2307/2872855',
 '10.2307/3723455',
 '10.2307/512967',
 '10.2307/40587680',
 '10.2307/438705',
 '10.2307/26158877',
 '10.2307/4501647',
 '10.2307/458780',
 '10.2307/459148',
 '10.2307/3736691',
 '10.2307/3728328',
 '10.2307/514110',
 '10.2307/433444',
 '10.2307/3717854',
 '10.2307/434174',
 '10.1086/598693',
 '10.2307/27654644',
 '10.2307/432736',
 '10.2307/437350',
 '10.2307/1261371',
 '10.2307/3730409',
 '10.2307/3715100',
 '10.2307/44016509',
 '10.2307/458990',
 '10.2307/468576',
 '10.2307/509276',
 '10.2307/457562',
 '10.2307/438895',
 '10.2307/3713225',
 '10.2307/3736356',
 '10.2307/511338',
 '10.2307/44016445',
 '10.1086/588108',
 '10.2307/20057704',
 '10.2307/3713138',
 '10.2307/510839',
 '10.2307/438632',
 '10.2307/3734330',
 '10.2307/1343470',
 '10.2307/3661532',
 '10.2307/3731247',
 '10.2307/508839',
 '10.2307/3727545',
 '10.2307/457644',
 '10.2307/2873185',
 '10.2307/436676',

In [60]:
DOI

,journal,year,authors,title,language,wordcount,doctype,doi,citation_counts,S2titles,S2years
0,The Review of English Studies,1969,[C. F. Williamson],Review Article,[eng],572,article | book-review,10.2307/512209,0,NA,0
1,Modern Philology,1979,[David Halliburton],Review Article,[eng],1863,article | book-review,10.2307/437950,0,NA,0
2,PMLA,1967,[Jean-Jacques Demorest],Pascal et le déséquilibre,[fre],4999,article | research-article,10.2307/461288,0,NA,0
3,The Review of English Studies,2003,[Robert Douglas-Fairhurst],Review Article,[eng],1140,article | book-review,10.2307/3661498,0,NA,0
4,The Modern Language Review,1951,[R. Weiss],Review Article,[eng],669,article | book-review,10.2307/3718589,0,NA,0


In [23]:
ret

[[None,
  'The Revisions of "Piers Plowman"',
  ['Art'],
  ['Mabel Day'],
  ['5345b72b54964d4009f83bfdc48a746c58a68d84'],
  ['S. Trigg'],
  1]]

In [68]:
out=pd.DataFrame(ret)

In [69]:
out.columns=["date", "title", "tags", "authors", "cited_ids", "cited_authors", "citation_count"]

In [71]:
out.to_csv("semantic_meta.csv")